In [1]:
import os
import sys
import matplotlib.pyplot as plt
from pyproj import CRS
from pyresample import geometry
import numpy as np


sys.path.insert(0, '/home/cameron/Projects/hypso-package')

# Import Satellite Object
from hypso import Hypso1

In [2]:
dir_path = '/home/cameron/Dokumenter/Data/frohavet'
nc_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-l1a.nc')
points_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-bin3.points')

In [3]:
satobj = Hypso1(hypso_path=nc_file, points_path=points_file, verbose=True)

satobj.generate_l1b_cube()

[INFO] Capture spatial dimensions: (598, 1092)
[INFO] Capture capture type: wide
[INFO] Running georeferencing...
No image mode provided. Detected image mode: bin3
[INFO] Running calibration routines...
[INFO] Running radiometric calibration...
[INFO] Running smile correction...
[INFO] Running destriping correction...


In [4]:
l1a_scene = satobj.get_l1a_satpy_scene()
l1b_scene = satobj.get_l1b_satpy_scene()

In [5]:
l1a_scene._datasets.keys()

[DataID(name='band_0'),
 DataID(name='band_1'),
 DataID(name='band_10'),
 DataID(name='band_100'),
 DataID(name='band_101'),
 DataID(name='band_102'),
 DataID(name='band_103'),
 DataID(name='band_104'),
 DataID(name='band_105'),
 DataID(name='band_106'),
 DataID(name='band_107'),
 DataID(name='band_108'),
 DataID(name='band_109'),
 DataID(name='band_11'),
 DataID(name='band_110'),
 DataID(name='band_111'),
 DataID(name='band_112'),
 DataID(name='band_113'),
 DataID(name='band_114'),
 DataID(name='band_115'),
 DataID(name='band_116'),
 DataID(name='band_117'),
 DataID(name='band_118'),
 DataID(name='band_119'),
 DataID(name='band_12'),
 DataID(name='band_13'),
 DataID(name='band_14'),
 DataID(name='band_15'),
 DataID(name='band_16'),
 DataID(name='band_17'),
 DataID(name='band_18'),
 DataID(name='band_19'),
 DataID(name='band_2'),
 DataID(name='band_20'),
 DataID(name='band_21'),
 DataID(name='band_22'),
 DataID(name='band_23'),
 DataID(name='band_24'),
 DataID(name='band_25'),
 DataID(

In [6]:
l1b_scene._datasets.keys()

[DataID(name='latitude'),
 DataID(name='longitude'),
 DataID(name='rad_387'),
 DataID(name='rad_391'),
 DataID(name='rad_394'),
 DataID(name='rad_398'),
 DataID(name='rad_402'),
 DataID(name='rad_405'),
 DataID(name='rad_409'),
 DataID(name='rad_412'),
 DataID(name='rad_416'),
 DataID(name='rad_419'),
 DataID(name='rad_423'),
 DataID(name='rad_426'),
 DataID(name='rad_430'),
 DataID(name='rad_433'),
 DataID(name='rad_437'),
 DataID(name='rad_441'),
 DataID(name='rad_444'),
 DataID(name='rad_448'),
 DataID(name='rad_451'),
 DataID(name='rad_455'),
 DataID(name='rad_458'),
 DataID(name='rad_462'),
 DataID(name='rad_465'),
 DataID(name='rad_469'),
 DataID(name='rad_472'),
 DataID(name='rad_476'),
 DataID(name='rad_479'),
 DataID(name='rad_483'),
 DataID(name='rad_486'),
 DataID(name='rad_490'),
 DataID(name='rad_494'),
 DataID(name='rad_497'),
 DataID(name='rad_501'),
 DataID(name='rad_504'),
 DataID(name='rad_508'),
 DataID(name='rad_511'),
 DataID(name='rad_515'),
 DataID(name='rad_518'